# Processing dataset from csv to parquet

In [1]:
import pyarrow.parquet as pq
import pandas as pd
import pyarrow as pa

In [18]:
# read S3 file into a data frame and show its data & metadata
df = pd.read_csv('../dataset/llama3.2/raw_dataset.csv')
df.head(30)

,text
0,### Human: Who are you? ### Assistant: I am Ja...
1,### Human: Introduce yourself. ### Assistant: ...
2,### Human: What is your purpose ### Assistant:...
3,### Human: What are you? ### Assistant: I am a...
4,### Human: Introduce yourself. ### Assistant: ...
5,### Human: What can you do for me? ### Assista...
6,### Human: How were you created? ### Assistant...
7,### Human: Brief me about your features. ### A...
8,### Human: Start by introducing yourself. ### ...
9,### Human: What is your primary function? ### ...


In [19]:
# Function to process the text and extract the conversations
def process_conversation(text):
    conversations = []
    # print(text)
    # Split the text into human and assistant parts
    parts = text.split("###")
    for part in parts:
        part = part.strip()
        if part.startswith("Human:"):
            value = part.replace("Human:", "").strip()
            conversations.append({"role": "user", "content": value})
        elif part.startswith("Assistant:"):
            value = part.replace("Assistant:", "").strip()
            conversations.append({"role": "assistant", "content": value})
    
    return conversations
# Apply the function to each row in the dataframe
df['conversations'] = df['text'].apply(process_conversation)

# Keep only the 'conversations' column as requested
df_final = df[['conversations']]

# [{"role": "user", "content": "Hi can I train llama model"}, {"role": "assistant", "content": "Sure"}]



# Display the processed dataframe to the user
# import ace_tools as tools; tools.display_dataframe_to_user(name="Processed Conversations", dataframe=df_final)

In [20]:
df_final.head(30)

,conversations
0,"[{'role': 'user', 'content': 'Who are you?'}, ..."
1,"[{'role': 'user', 'content': 'Introduce yourse..."
2,"[{'role': 'user', 'content': 'What is your pur..."
3,"[{'role': 'user', 'content': 'What are you?'},..."
4,"[{'role': 'user', 'content': 'Introduce yourse..."
5,"[{'role': 'user', 'content': 'What can you do ..."
6,"[{'role': 'user', 'content': 'How were you cre..."
7,"[{'role': 'user', 'content': 'Brief me about y..."
8,"[{'role': 'user', 'content': 'Start by introdu..."
9,"[{'role': 'user', 'content': 'What is your pri..."


In [54]:
from flex_ai import FlexAI, DatasetType
client = FlexAI(api_key="24c3f04e-ad2c-4eec-afac-70d4dad61e85")

# lloop through a dictionary
for index, row in df_final.iterrows():
    # print(row['conversations'])
    # itterate and save to train.jsonl
    all_conversations.append(row['conversations'])

# print(all_conversations)
# save all_conversations to train.jsonl
import jsonlines
with jsonlines.open('train.jsonl', mode='w') as writer:
    writer.write_all(all_conversations)

with jsonlines.open('eval.jsonl', mode='w') as writer:
    writer.write_all(all_conversations)


In [56]:
dataset = client.create_dataset("API Dataset New",
                               "train.jsonl",
                               "eval.jsonl",
                                DatasetType.CHAT)


Map: 100%|██████████| 1150/1150 [00:00<00:00, 25730.42 examples/s]


Train dataset uploaded successfully.
Eval dataset uploaded successfully.
New Dataset created successfully.
{
    "created_at": "2024-10-07T21:41:43.072546+00:00",
    "eval_path": "8e92ae6f-91ca-4f21-bc3e-af1ffdb048c8/94c15559-7fb2-4d96-8128-7d45548bb080/eval.jsonl",
    "eval_rows_count": 1150,
    "id": "94c15559-7fb2-4d96-8128-7d45548bb080",
    "max_tokens": 97,
    "name": "API Dataset New",
    "storage_type": "FLEX",
    "total_tokens": 51325,
    "train_path": "8e92ae6f-91ca-4f21-bc3e-af1ffdb048c8/94c15559-7fb2-4d96-8128-7d45548bb080/train.jsonl",
    "train_rows_count": 1150,
    "type": "CHAT",
    "updated_at": "2024-10-07T21:41:43.072546+00:00",
    "user_id": "8e92ae6f-91ca-4f21-bc3e-af1ffdb048c8"
}
